In [32]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [33]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [34]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [35]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 18355.82it/s]


In [36]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 134.95it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [37]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [38]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [39]:
data = kernelclass.calc_kernel_mtxel()

In [40]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.47881-0.00000j | -0.02994-0.11418j | -0.07084-0.02386j | -0.06933+0.12470j | -0.10282+0.15727j | 0.01616-0.05548j | 0.02306+0.09088j | 0.00050-0.00007j
-0.02994-0.11418j | 0.21656+0.00000j | 0.02322-0.03526j | -0.01349-0.01293j | 0.02786+0.03082j | -0.01573-0.00945j | -0.10018-0.00079j | -0.00211+0.01527j
-0.07084-0.02386j | 0.02322-0.03526j | 0.21656+0.00000j | 0.00335-0.01821j | -0.01061+0.04092j | 0.00415+0.10010j | 0.01008+0.01557j | 0.00049-0.00024j
-0.06933+0.12470j | -0.01349-0.01293j | 0.00335-0.01821j | 0.24228-0.00000j | 0.09993+0.00717j | 0.00642-0.00146j | -0.00472+0.00445j | 0.05010+0.06913j
-0.10282+0.15727j | 0.02786+0.03082j | -0.01061+0.04092j | 0.09993+0.00717j | 0.21656+0.00000j | -0.01754+0.00533j | 0.01264-0.01377j | -0.00966-0.01149j
0.01616-0.05548j | -0.01573-0.00945j | 0.00415+0.10010j | 0.00642-0.00146j | -0.01754+0.00533j | 0.24235+0.00000j | -0.00570+0.00336j | 0.00109+0.00256j
0.02306+0.09088j | -0.10018-0.00079j | 0.01008+0.01557j | -0

In [41]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.05016-0.19131j | -0.11326-0.03814j | -0.12329+0.21733j | -0.17991+0.27518j | 0.02843-0.09821j | 0.04308+0.15990j | -0.00000+0.00000j
-0.05016-0.19131j | 1.00000+0.00000j | 0.04481-0.06806j | -0.24097-0.23319j | 0.05318+0.05884j | -0.28182-0.17010j | -0.03101+0.00090j | -0.00024+0.00175j
-0.11326-0.03814j | 0.04481-0.06806j | 1.00000-0.00000j | 0.06072-0.32644j | -0.02049+0.07906j | 0.00142+0.03097j | 0.18195+0.27839j | 0.00017-0.00008j
-0.12329+0.21733j | -0.24097-0.23319j | 0.06072-0.32644j | 1.00000-0.00000j | 0.03087+0.00288j | -0.04309+0.00855j | 0.03204-0.02999j | 0.01672+0.02048j
-0.17991+0.27518j | 0.05318+0.05884j | -0.02049+0.07906j | 0.03087+0.00288j | 1.00000+0.00000j | -0.31441+0.09672j | 0.22525-0.24866j | -0.00121-0.00144j
0.02843-0.09821j | -0.28182-0.17010j | 0.00142+0.03097j | -0.04309+0.00855j | -0.31441+0.09672j | 1.00000+0.00000j | 0.03747-0.02293j | 0.00345-0.00052j
0.04308+0.15990j | -0.03101+0.00090j | 0.18195+0.27839j | 0.03204-0.02999j | 0

In [42]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | -0.00163-0.00622j | -0.03364-0.01133j | 0.00084-0.00162j | -0.00289+0.00443j | -0.00897+0.03042j | -0.00125-0.00600j | -0.00050+0.00006j
-0.00163-0.00622j | 0.00000+0.00000j | 0.00153-0.00232j | -0.07397-0.07119j | -0.00419-0.00463j | -0.23463-0.14122j | -0.00059+0.00006j | 0.00013-0.00092j
-0.03364-0.01133j | 0.00153-0.00232j | 0.00000+0.00000j | -0.01094+0.05630j | -0.00340+0.01312j | 0.00036+0.00393j | 0.09960+0.15410j | 0.00060-0.00028j
0.00084-0.00162j | -0.07397-0.07119j | -0.01094+0.05630j | 0.00000+0.00000j | 0.00015-0.00001j | -0.00685+0.00171j | 0.00157-0.00136j | -0.01041-0.01424j
-0.00289+0.00443j | -0.00419-0.00463j | -0.00340+0.01312j | 0.00015-0.00001j | 0.00000+0.00000j | 0.09009-0.02728j | 0.07158-0.07859j | -0.00016-0.00019j
-0.00897+0.03042j | -0.23463-0.14122j | 0.00036+0.00393j | -0.00685+0.00171j | 0.09009-0.02728j | 0.00000+0.00000j | -0.00016+0.00020j | -0.00017-0.00088j
-0.00125-0.00600j | -0.00059+0.00006j | 0.09960+0.15410j | 0.00157-0.0013

In [43]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.31795+0.00000j | -0.06968-0.26587j | -0.12001-0.04035j | -0.19387+0.35108j | -0.31236+0.47773j | 0.03263-0.11362j | 0.05342+0.21388j | 0.00698-0.00084j
-0.06968-0.26587j | 0.12454-0.00000j | 0.17075-0.25933j | -0.23568-0.22403j | 0.13847+0.15321j | -0.44811-0.26727j | -0.06415-0.00212j | -0.01214+0.08798j
-0.12001-0.04035j | 0.17075-0.25933j | 0.11524-0.00000j | 0.06234-0.34520j | -0.07107+0.27426j | 0.00240+0.06594j | 0.27653+0.43032j | 0.00915-0.00437j
-0.19387+0.35108j | -0.23568-0.22403j | 0.06234-0.34520j | 0.01506-0.00006j | 0.05347+0.00319j | 0.08703-0.01773j | -0.01576+0.01491j | 0.02838+0.04275j
-0.31236+0.47773j | 0.13847+0.15321j | -0.07107+0.27426j | 0.05347+0.00319j | 0.15507+0.00000j | -0.59121+0.17740j | 0.35025-0.37856j | -0.03355-0.03993j
0.03263-0.11362j | -0.44811-0.26727j | 0.00240+0.06594j | 0.08703-0.01773j | -0.59121+0.17740j | 0.02686+0.00086j | -0.11239+0.06635j | -0.01039-0.01086j
0.05342+0.21388j | -0.06415-0.00212j | 0.27653+0.43032j | -0.01576+0.01491j | 

In [44]:
l_epsinv = epsilon.l_epsinv
eps0mat = l_epsinv[0]

# for row in range(0, eps0mat.shape[0]):
#     for col in range(0, eps0mat.shape[1]):
#         print(f"eps0mat[{row}, {col}] = {eps0mat[row, col]}")


In [45]:
l_epsinv_sorted = kernelclass.l_epsinv
eps0mat_sorted = l_epsinv_sorted[0]

# for row in range(0, eps0mat_sorted.shape[0]):
#     for col in range(0, eps0mat_sorted.shape[1]):
#         print(f"eps0mat_sorted[{row}, {col}] = {eps0mat_sorted[row, col]}")

In [46]:
head = np.real(head)
wings = np.real(wings)
body = np.real(body)
exc = np.real(exc)

In [47]:
import numpy as np

head_from_BSE = np.loadtxt(dirname + 'head.csv', delimiter=',')
wings_from_BSE = np.loadtxt(dirname + 'wing.csv', delimiter=',')
exc_from_BSE = np.loadtxt(dirname + 'exchange.csv', delimiter=',')
body_from_BSE = np.loadtxt(dirname + 'body.csv', delimiter=',')

In [48]:
# print the sum of abs**2 of the elements of these matrices

sum_abs_head = np.sum(np.abs(head)**2)
sum_abs_wings = np.sum(np.abs(wings)**2)
sum_abs_exc = np.sum(np.abs(exc)**2)
sum_abs_body = np.sum(np.abs(body)**2)

print(f"Sum of abs**2 of head: {sum_abs_head}")
print(f"Sum of abs**2 of wings: {sum_abs_wings}")
print(f"Sum of abs**2 of exc: {sum_abs_exc}")
print(f"Sum of abs**2 of body: {sum_abs_body}")

# Compare the results with BGW's results
sum_abs_head_from_BSE = np.sum(np.abs(head_from_BSE)**2)
sum_abs_wings_from_BSE = np.sum(np.abs(wings_from_BSE)**2)
sum_abs_exc_from_BSE = np.sum(np.abs(exc_from_BSE)**2)
sum_abs_body_from_BSE = np.sum(np.abs(body_from_BSE)**2)

print(f"Sum of abs**2 of head from BSE: {sum_abs_head_from_BSE}")
print(f"Sum of abs**2 of wings from BSE: {sum_abs_wings_from_BSE}")
print(f"Sum of abs**2 of exc from BSE: {sum_abs_exc_from_BSE}")
print(f"Sum of abs**2 of body from BSE: {sum_abs_body_from_BSE}")


# Calculate the sum of abs**2 of the difference between the two matrices
diff_head = head - head_from_BSE
diff_wings = wings - wings_from_BSE 
diff_exc = exc - exc_from_BSE
diff_body = body - body_from_BSE

sum_abs_diff_head = np.sum(np.abs(diff_head)**2)
sum_abs_diff_wings = np.sum(np.abs(diff_wings)**2)
sum_abs_diff_exc = np.sum(np.abs(diff_exc)**2)
sum_abs_diff_body = np.sum(np.abs(diff_body)**2)

print(f"Sum of abs**2 of diff head: {sum_abs_diff_head}")
print(f"Sum of abs**2 of diff wings: {sum_abs_diff_wings}")
print(f"Sum of abs**2 of diff exc: {sum_abs_diff_exc}")
print(f"Sum of abs**2 of diff body: {sum_abs_diff_body}")

Sum of abs**2 of head: 8.802449312551603
Sum of abs**2 of wings: 0.17043935309004027
Sum of abs**2 of exc: 0.6878312737107231
Sum of abs**2 of body: 2.270600917439955
Sum of abs**2 of head from BSE: 8.802449312551602
Sum of abs**2 of wings from BSE: 0.20433951118898783
Sum of abs**2 of exc from BSE: 0.6878312737107226
Sum of abs**2 of body from BSE: 2.316625974925835
Sum of abs**2 of diff head: 6.944320930633691e-31
Sum of abs**2 of diff wings: 0.17842851722135442
Sum of abs**2 of diff exc: 1.4041248282224752e-31
Sum of abs**2 of diff body: 0.2578551472334631
